In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import cufflinks as cf  
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

cf.go_offline()

In [11]:
liver_patient = pd.read_csv("duomenys/Indian_Liver_Patient.csv")
liver_patient.head()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [12]:
#pervadiname stulpelį dataset į liver_disease
liver_patient.rename(columns={'Dataset': 'Liver_Disease'}, inplace=True)
liver_patient['Gender'] = liver_patient['Gender'].apply(lambda x: 1 if x =='Male' else 0)
#Female=0 and Male = 1
liver_patient['Albumin_and_Globulin_Ratio'].mean()
liver_patient=liver_patient.fillna(0.94)

In [13]:
liver_patient.groupby('Liver_Disease').size()

Liver_Disease
1    416
2    167
dtype: int64

In [14]:
liver_patient.columns # Pasižiūrime visų stulpelių pavadinimus

Index(['Age', 'Gender', 'Total_Bilirubin', 'Direct_Bilirubin',
       'Alkaline_Phosphotase', 'Alamine_Aminotransferase',
       'Aspartate_Aminotransferase', 'Total_Protiens', 'Albumin',
       'Albumin_and_Globulin_Ratio', 'Liver_Disease'],
      dtype='object')

In [16]:
# Pasidarome X ir Y, kurį prognozuosime
X = liver_patient[['Age', 'Gender', 'Total_Bilirubin', 'Direct_Bilirubin',
       'Alkaline_Phosphotase', 'Alamine_Aminotransferase',
       'Aspartate_Aminotransferase', 'Total_Protiens', 'Albumin',
       'Albumin_and_Globulin_Ratio']]
Y = liver_patient['Liver_Disease']

In [17]:
# Pasidarome test ir train rinkinius
mask = np.random.random(liver_patient.shape[0]) < 0.7

In [21]:
model = LogisticRegression()

model.fit(liver_patient[mask].drop('Liver_Disease', axis=1), liver_patient[mask]['Liver_Disease'])
pred = model.predict(liver_patient[~mask].drop('Liver_Disease', axis=1))
prob = model.predict_proba(liver_patient[~mask].drop('Liver_Disease', axis=1))

In [22]:
print('Log-loss = {0:.02f}'.format(
    log_loss(liver_patient[~mask]['Liver_Disease'], prob)))
print('Hit rate = {0:.02%}'.format(
    (liver_patient[~mask]['Liver_Disease'] == pred).mean()))

Log-loss = 0.51
Hit rate = 71.43%


In [ ]:
# Reikia dar pabandyti pridėti kokių nors išvestinių stulpelių ir pagerinti Log-Loss rodiklį

# Design metrics, which improve model score

In [28]:
#Apsiskaiciuojame Globulina
liver_patient['Globulin'] = liver_patient['Albumin'] / liver_patient['Albumin_and_Globulin_Ratio']

#Susiskaiciuojam netiesiogini bilirubina (nekonjuguotas) (jis lygus is bendro atemus tiesiogini)
liver_patient['Indirect_Bilirubin'] = liver_patient['Total_Bilirubin'] - liver_patient['Direct_Bilirubin']
liver_patient.head()


,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Liver_Disease,Indirect_Bilirubin,Globulin
0,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90,1,0.6,3.666667
1,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74,1,5.4,4.324324
2,62,1,7.3,4.1,490,60,68,7.0,3.3,0.89,1,3.2,3.707865
3,58,1,1.0,0.4,182,14,20,6.8,3.4,1.00,1,0.6,3.400000
4,72,1,3.9,2.0,195,27,59,7.3,2.4,0.40,1,1.9,6.000000


In [29]:
model2 = LogisticRegression()

model2.fit(liver_patient[mask].drop('Liver_Disease', axis=1), liver_patient[mask]['Liver_Disease'])
pred2 = model2.predict(liver_patient[~mask].drop('Liver_Disease', axis=1))
prob2 = model2.predict_proba(liver_patient[~mask].drop('Liver_Disease', axis=1))

print('Log-loss = {0:.02f}'.format(
    log_loss(liver_patient[~mask]['Liver_Disease'], prob2)))
print('Hit rate = {0:.02%}'.format(
    (liver_patient[~mask]['Liver_Disease'] == pred2).mean()))

Log-loss = 0.51
Hit rate = 73.02%
